# 📦 Kaggle → Google Drive Backup

**Ce notebook télécharge tous vos notebooks Kaggle et les sauvegarde sur Google Drive.**

---

## Prérequis
1. Téléchargez votre fichier `kaggle.json` depuis: https://www.kaggle.com/settings (section API)
2. Exécutez les cellules dans l'ordre

In [ ]:
# === Étape 1: Installation ===
!pip install kaggle -q
print("✓ Kaggle API installée")

In [ ]:
# === Étape 2: Upload de votre kaggle.json ===
# Cliquez sur 'Choose Files' et sélectionnez votre kaggle.json
from google.colab import files
import os

print("📤 Uploadez votre fichier kaggle.json:")
print("   (Téléchargez-le depuis https://www.kaggle.com/settings → API → Create New Token)")
print()

uploaded = files.upload()

# Configure Kaggle credentials
os.makedirs(os.path.expanduser('~/.kaggle'), exist_ok=True)
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print()
print("✓ Credentials Kaggle configurées!")

In [ ]:
# === Étape 3: Lister vos notebooks Kaggle ===
import subprocess
import json

# Vos usernames Kaggle (vous avez 2 comptes)
KAGGLE_USERNAMES = ['dominiqueloyer2', 'dominiquesloyer']

all_notebooks = []

for username in KAGGLE_USERNAMES:
    print(f"\n📋 Notebooks de {username}:")
    print("=" * 40)
    
    try:
        result = subprocess.run(
            ['kaggle', 'kernels', 'list', '--user', username, '--page-size', '100'],
            capture_output=True, text=True
        )
        print(result.stdout)
        
        # Parse output to get notebook slugs
        lines = result.stdout.strip().split('\n')
        for line in lines[1:]:  # Skip header
            if line.strip():
                parts = line.split()
                if parts:
                    slug = parts[0]
                    all_notebooks.append((username, slug))
    except Exception as e:
        print(f"Erreur: {e}")

print(f"\n📊 Total: {len(all_notebooks)} notebooks trouvés")

In [ ]:
# === Étape 4: Monter Google Drive ===
from google.colab import drive

print("🔗 Connexion à Google Drive...")
print("   → Connectez-vous avec thisisminebasicallys@gmail.com (2 TO)")
print()

drive.mount('/content/drive')

# Créer le dossier de backup
BACKUP_DIR = '/content/drive/MyDrive/Kaggle_Notebooks_Backup'
os.makedirs(BACKUP_DIR, exist_ok=True)

print(f"\n✓ Drive monté!")
print(f"✓ Dossier de backup: {BACKUP_DIR}")

In [ ]:
# === Étape 5: Télécharger tous les notebooks ===
import time

print("📥 Téléchargement des notebooks...")
print("=" * 50)

# Dossier temporaire
TEMP_DIR = '/content/kaggle_downloads'
os.makedirs(TEMP_DIR, exist_ok=True)

success_count = 0
failed = []

for i, (username, slug) in enumerate(all_notebooks, 1):
    notebook_name = slug.split('/')[-1] if '/' in slug else slug
    print(f"[{i}/{len(all_notebooks)}] {slug}...", end=" ")
    
    try:
        # Télécharger le notebook
        result = subprocess.run(
            ['kaggle', 'kernels', 'pull', slug, '-p', TEMP_DIR],
            capture_output=True, text=True, timeout=60
        )
        
        if result.returncode == 0:
            print("✓")
            success_count += 1
        else:
            print(f"⚠ {result.stderr[:50]}")
            failed.append(slug)
            
    except subprocess.TimeoutExpired:
        print("⏱ Timeout")
        failed.append(slug)
    except Exception as e:
        print(f"❌ {str(e)[:30]}")
        failed.append(slug)
    
    # Pause entre les téléchargements pour éviter les rate limits
    time.sleep(1)

print()
print("=" * 50)
print(f"✓ Téléchargés: {success_count}/{len(all_notebooks)}")
if failed:
    print(f"⚠ Échecs: {len(failed)}")
    for f in failed:
        print(f"   - {f}")

In [ ]:
# === Étape 6: Copier vers Google Drive ===
import shutil
from datetime import datetime

# Lister les fichiers téléchargés
downloaded_files = os.listdir(TEMP_DIR)
print(f"📁 {len(downloaded_files)} fichiers à copier:")

for filename in downloaded_files:
    src = os.path.join(TEMP_DIR, filename)
    dst = os.path.join(BACKUP_DIR, filename)
    
    print(f"  📄 {filename}...", end=" ")
    try:
        shutil.copy2(src, dst)
        print("✓")
    except Exception as e:
        print(f"❌ {e}")

print()
print("=" * 50)
print(f"✅ BACKUP TERMINÉ!")
print(f"📍 Emplacement: Google Drive > Kaggle_Notebooks_Backup")
print(f"📅 Date: {datetime.now().strftime('%Y-%m-%d %H:%M')}")

In [ ]:
# === Vérification finale ===
print("📋 Contenu du dossier de backup:")
print("=" * 50)

backup_files = os.listdir(BACKUP_DIR)
total_size = 0

for f in sorted(backup_files):
    path = os.path.join(BACKUP_DIR, f)
    size = os.path.getsize(path)
    total_size += size
    print(f"  {f} ({size/1024:.1f} KB)")

print()
print(f"📊 Total: {len(backup_files)} fichiers, {total_size/1024/1024:.2f} MB")